#### Instruction (Read this)
- Use this template to develop your project. Do not change the steps. 
- For each step, you may add additional cells if needed.
- But remove <b>unnecessary</b> cells to ensure the notebook is readable.
- Marks will be <b>deducted</b> if the notebook is cluttered or difficult to follow due to excess or irrelevant content.
- <b>Briefly</b> describe the steps in the "Description:" field.
- <b>Do not</b> submit the dataset. 
- The submitted jupyter notebook will be executed using the uploaded dataset in eLearn.

#### Group Information

Group No: Seismic 6

- Member 1: Lim Wen Hao
- Member 2: Goey Chew Hong
- Member 3: Lim Cong Sheng
- Member 4: Tan Jun Cheng


#### Import libraries

In [ ]:
%config Completer.use_jedi=False # comment if not needed

# Import necessary libraries for data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries for machine learning
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier # for KNN
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text # for decision tree

# Import necessary libraries for Feature Selection and Sampling
from imblearn.over_sampling import SMOTE # for oversampling
from collections import Counter # for counting

%matplotlib inline

#### Load the dataset

From the dataset README documentation, 

**Attribute information:**
1. `seismic`: result of shift seismic hazard assessment in the mine working obtained by the seismic method (a - lack of hazard, b - low hazard, c - high hazard, d - danger state);
2. `seismoacoustic`: result of shift seismic hazard assessment in the mine working obtained by the seismoacoustic method;
3. `shift`: information about type of a shift (W - coal-getting, N -preparation shift);
4. `genergy`: seismic energy recorded within previous shift by the most active geophone (GMax) out of geophones monitoring the longwall;
5. `gpuls`: a number of pulses recorded within previous shift by GMax;
6. `gdenergy`: a deviation of energy recorded within previous shift by GMax from average energy recorded during eight previous shifts;
7. `gdpuls`: a deviation of a number of pulses recorded within previous shift by GMax from average number of pulses recorded during eight previous shifts;
8. `ghazard`: result of shift seismic hazard assessment in the mine working obtained by the seismoacoustic method based on registration coming form GMax only;
9. `nbumps`: the number of seismic bumps recorded within previous shift;
10. `nbumps2`: the number of seismic bumps (in energy range [10^2,10^3)) registered within previous shift;
11. `nbumps3`: the number of seismic bumps (in energy range [10^3,10^4)) registered within previous shift;
12. `nbumps4`: the number of seismic bumps (in energy range [10^4,10^5)) registered within previous shift;
13. `nbumps5`: the number of seismic bumps (in energy range [10^5,10^6)) registered within the last shift;
14. `nbumps6`: the number of seismic bumps (in energy range [10^6,10^7)) registered within previous shift;
15. `nbumps7`: the number of seismic bumps (in energy range [10^7,10^8)) registered within previous shift;
16. `nbumps89`: the number of seismic bumps (in energy range [10^8,10^10)) registered within previous shift;
17. `energy`: total energy of seismic bumps registered within previous shift;
18. `maxenergy`: the maximum energy of the seismic bumps registered within previous shift;
19. `class`: the decision attribute - "1" means that high energy seismic bump occurred in the next shift 

In [ ]:
df = pd.read_csv("seismic-bumps.csv")
df.sample(10)

From the sample we got, `class` column consists only 0 and 1 which is the target label column.

In [ ]:
# Summarize the dataset
df.info()

In [ ]:
# Check for missing values
df.isnull().sum()

From the summary we get,
1. No null values found in all columns
2. There are 19 columns and 2584 records found in the dataset.

In [ ]:
# Set the target variable
label = "class"
df[label].value_counts()

In [ ]:
# Categorize the column based on data types
categorical_df = df.select_dtypes(include='object')
categorical_df.head()

In [ ]:
# Categorize the column based on data types
numerical_df = df.select_dtypes(include='int64')
numerical_df.head()

From the value counts, 2414 records found with `class` = 0, and 170 records of `class` = 1.

In [ ]:
# Plot the correlation matrix for numerical features
corr = numerical_df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

From the correlation matrix, among the features that highly correlated with each others include:
- `energy` and `maxenergy` (0.99)
- `gdpuls` and `gdenergy` (0.81)
- `nbumps5` and `maxenergy` (0.81)
- `nbumps` and `nbumps2` (0.8)
- `nbumps` and `nbumps3` (0.8)
- `nbumps5` and `energy` (0.77)
- `gpuls` and `genergy` (0.75)

Features that do not show any correlation with other classes:
- `nbumps6`
- `nbumps7`
- `nbumps89`

These features can be dropped in the subsequent steps as they do not need to be involved in the model training.

**None of the features** show direct correlation with the target column, `class`.

`class` column can be dropped from numerical DataFrame `numerical_df` as it is a target column

In [ ]:
# Drop the features that are not needed for the analysis
df.drop(['nbumps89', 'nbumps7', 'nbumps6'], axis=1, inplace=True)
df.head()

In [ ]:
# Drop the features that are not needed for the analysis and the target variable
numerical_df.drop(['nbumps89', 'nbumps7', 'nbumps6', 'class'], axis=1, inplace=True)
numerical_df.head()

In [ ]:
# Summarize the numerical features
numerical_df.describe()

In [ ]:
# Categorize the continuous features
continuous_df = numerical_df[["genergy", "gpuls", "gdenergy", "gdpuls", "energy", "maxenergy"]]
continuous_df.head()

In [ ]:
# Plot the distribution of continuous features
sns.pairplot(continuous_df, size = 2.5)
plt.show()

In [ ]:
# Set the figure size
plt.figure(figsize=(20, 15))

# Loop through the columns and create subplots
for i, column in enumerate(continuous_df.columns, 1):
  plt.subplot(2, 3, i)  # Create a subplot in a 2x3 grid
  fig = df.boxplot(column=column)
  fig.set_title('')
  fig.set_ylabel(column)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# remove the duplicated rows
df.drop_duplicates(keep='first', inplace=True)
df

In [ ]:
# Reindex the DataFrame after dropping duplicates
df.reset_index(drop=True, inplace=True)
df.tail()

In [ ]:
# Check for the dimensions of the DataFrame after data cleaning
df.shape

In [ ]:
# Save the cleaned DataFrame
X = df.drop(columns=label).values
y = df[label].values

# Show the shape of the data
print("X shape: ", X.shape)
print("y shape: ", y.shape)

#### Split the dataset
Split the dataset into training, validation and test sets.

In [ ]:
# Split the dataset into training, validation, and test sets
# Training : Validation : Test = 7 : 1 : 2

seed_num = 42
np.random.seed(seed_num)

# Split the dataset into training and test sets
# test_size = 0.2 as (training dataset + validation dataset) : test dataset = 8 : 2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_num,
  # stratify=y
)

# Split the training dataset into training and validation sets
# test_size = 0.125 as training dataset : validation dataset = 7 : 1
X_train, X_vald, y_train, y_vald = train_test_split(
  X_train,
  y_train,
  test_size=0.125,
  random_state=seed_num,
  # stratify=y_train
)

# Show the shape of the datasets
print("Original dataset shape:")
print("X shape:", X.shape, ", y shape:", y.shape, "\n")

print("Training set shape:")
print("X_train shape:", X_train.shape, ", y_train shape:", y_train.shape, "\n")

print("Validation set shape:")
print("X_vald shape:", X_vald.shape, ", y_vald shape:", y_vald.shape, "\n")

print("Test set shape: ")
print("X_test shape:", X_test.shape, ", y_test shape:", y_test.shape)

#### Data preprocessing
Perform data preprocessing such as normalization, standardization, label encoding etc.
______________________________________________________________________________________
Description:

This section outlines a comprehensive data preprocessing pipeline designed to prepare the Seismic Bumps Dataset for a classification task aimed at predicting whether a high-energy seismic bump will occur in the next shift.

The process includes **label encoding and data scaling**: ordinal categorical features are transformed using `Label Encoding,` while nominal features are encoded with `One-Hot Encoding`. Numerical features are scaled using the `Robust Scaler` to reduce the impact of outliers and skewed distributions. To address class imbalance, the `SMOTE (Synthetic Minority Over-sampling Technique)` is applied exclusively to the training set.

##### Label encoding & data scaling

Based on the column description, DataFrame consists of

Categorical data includes
- Ordinal: `seismic`, `seismoacoustic`, and `ghazard`
- Nominal: `shift`

and numerical data.

The data are encoded based on their categories:
- Ordinal categorical values using **Label Encoding**,
- Nominal categorical values using **One Hot Encoding**,
- and Numerical values using **Robust Scaler**

**Robust Scaler** is used instead of **MinMaxScaler** as seismic data often includes extreme value (e.g. sudden energy spikes, zero-heavy counts). RoubstScaler is typically safer as it is good for data that has outliers or heavy skew.

In [ ]:
# Convert X_train to a DataFrame for scaling
X_train_df = pd.DataFrame(X_train, columns=df.drop(columns=label).columns)
X_vald_df = pd.DataFrame(X_vald, columns=df.drop(columns=label).columns)
X_test_df = pd.DataFrame(X_test, columns=df.drop(columns=label).columns)

In [ ]:
# Apply Label Encoding to the ordinal categorical columns
label_encoder = LabelEncoder()

for column in categorical_df.columns:
  if column != 'shift':
    # Transform the training, validation, and test sets
    X_train_df[column] = label_encoder.fit_transform(X_train_df[column])
    X_vald_df[column] = label_encoder.transform(X_vald_df[column])
    X_test_df[column] = label_encoder.transform(X_test_df[column])

In [ ]:
# Apply One-Hot Encoding to the nominal categorical columns
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

def onehot_encode_and_concat(df, column, onehot_encoder): 
  onehot_encoded = onehot_encoder.transform(df[[column]])
  onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out([column]), index=df.index)
  df = df.drop(column, axis=1) # drop the original column
  return pd.concat([df, onehot_df], axis=1)

onehot_encoder.fit(X_train_df[['shift']])

# Transform the training, validation, and test sets
X_train_df = onehot_encode_and_concat(X_train_df, 'shift', onehot_encoder)
X_vald_df = onehot_encode_and_concat(X_vald_df, 'shift', onehot_encoder)
X_test_df = onehot_encode_and_concat(X_test_df, 'shift', onehot_encoder)

In [ ]:
# Apply Robust Scaling to the numerical columns
numerical_scaler = RobustScaler()

# Transform the training, validation, and test sets
X_train_df[numerical_df.columns] = numerical_scaler.fit_transform(X_train_df[numerical_df.columns])
X_vald_df[numerical_df.columns] = numerical_scaler.transform(X_vald_df[numerical_df.columns])
X_test_df[numerical_df.columns] = numerical_scaler.transform(X_test_df[numerical_df.columns])

In [ ]:
# Checkpoint the columns after encoding
columns_after_encoding = X_train_df.columns
columns_after_encoding

In [ ]:
# Display the scaled columns of the training set
X_train_df[numerical_df.columns].head()

In [ ]:
# Display the scaled columns of the validation set
X_vald_df[numerical_df.columns].head()

In [ ]:
# Display the scaled columns of the test set
X_test_df[numerical_df.columns].head()

##### Oversampling
**SMOTE (Synthetic Minority Over-sampling Technique)**, is used for oversampling the minority class in imbalanced datasets. It is only introduced to training dataset to prevent the unrealistic samples and data leakage in validation dataset and test dataset.

In [ ]:
# Oversampling
# SMOTE
smote = SMOTE(random_state=seed_num, sampling_strategy='minority')
X_smote, y_smote = smote.fit_resample(X_train_df, y_train)

print('Original dataset shape:', Counter(y_train))
print('Resample dataset shape:', Counter(y_smote))

X_train = X_smote
y_train = y_smote

In [ ]:
# Convert back to a NumPy array
X_vald = X_vald_df.values
X_test = X_test_df.values

#### Feature Selection
Perform feature selection to select the relevant features.
______________________________________________________________________________________
Description:

Feature selection is then performed using the `SelectKBest` method with mutual information to retain the eight most informative variables.

The final output consists of clean, balanced, and scaled datasets, ready for model development and evaluation.

In [ ]:
# Feature selection
selector = SelectKBest(score_func=mutual_info_classif, k=8)
X_train = selector.fit_transform(X_train, y_train)

# Transform the validation and test sets using the same selector
X_vald = selector.transform(X_vald)
X_test = selector.transform(X_test)

# Get the feature names after encoding
selected_indices = selector.get_support(indices=True) # Get the indices of the selected features
selected_features = columns_after_encoding[selected_indices] # Get the names of the selected features
selected_features

In [ ]:
# Get all feature names BEFORE selection
all_feature_names = X_train_df.columns  # assuming x_train is a DataFrame

# Get all feature scores from the selector
all_scores = selector.scores_  # same length as original feature count

# Create a DataFrame of all features and their scores
feature_importances = pd.DataFrame({
  "feature": all_feature_names,
  "importance": all_scores
})

# Sort the DataFrame by importance
feature_importances = feature_importances.sort_values("importance", ascending=False)

# Plot all features
plt.figure(figsize=(12, max(6, len(feature_importances) * 0.3)))  # dynamic height
sns.barplot(x="importance", y="feature", data=feature_importances)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance (All Features)")
plt.tight_layout()
plt.show()


In [ ]:
# Get the feature importances
# Get the indices of the selected features
importances = selector.scores_[selected_indices] # Get the scores of the selected features
feature_importances = pd.DataFrame({"feature": selected_features, "importance": importances}) # Create a DataFrame with feature names and their importances
feature_importances = feature_importances.sort_values("importance", ascending=False) # Sort the DataFrame by importance

plt.figure(figsize=(10, 8))
sns.barplot(x=feature_importances['importance'], y=feature_importances['feature'])
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importance (Selected Features)')
plt.show()

In [ ]:
# Store the selected features back into the training set
X_train_df = pd.DataFrame(X_train, columns=selected_features)
X_train_df.head()

In [ ]:
# Store the selected features back into the validation set
X_vald_df = pd.DataFrame(X_vald, columns=selected_features)
X_vald_df.head()

In [ ]:
# Store the selected features back into the test set
X_test_df = pd.DataFrame(X_test, columns=selected_features)
X_test_df.head()

In [ ]:
# Convert back to a NumPy array
X_train = X_train_df.values
X_vald = X_vald_df.values
X_test = X_test_df.values

In [ ]:
# Check the shape of the datasets after oversampling
print("Training set shape:")
print("X_train shape:", X_train.shape, ", y_train shape:", y_train.shape)
print("X_train_df shape:", X_train_df.shape, ", y_train shape:", y_train.shape, "\n")

print("Validation set shape:")
print("X_vald shape:", X_vald.shape, ", y_vald shape:", y_vald.shape)
print("X_vald_df shape:", X_vald_df.shape, ", y_vald shape:", y_vald.shape, "\n")

print("Test set shape: ")
print("X_test shape:", X_test.shape, ", y_test shape:", y_test.shape)
print("X_test_df shape:", X_test_df.shape, ", y_test shape:", y_test.shape)

#### Data modeling
Build the machine learning models. You must build atleast two (2) predictive models. One of the predictive models must be either Decision Tree or Support Vector Machine.
______________________________________________________________________________________
Description:

##### Variables & Functions

In [ ]:
# Repeated Stratified K-Fold Cross Validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=seed_num)

In [ ]:
def plot_confusion_matrices_side_by_side(y_true, y_pred_default, y_pred_optimized, model_name):
    """
    Plot confusion matrices for two models side by side
    """
    cm_default = confusion_matrix(y_true, y_pred_default)
    cm_optimized = confusion_matrix(y_true, y_pred_optimized)

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Default model
    sns.heatmap(cm_default, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title(f'Confusion Matrix - Default {model_name}')
    axes[0].set_xlabel('Predicted Label')
    axes[0].set_ylabel('True Label')

    # Optimized model
    sns.heatmap(cm_optimized, annot=True, fmt='d', cmap='Blues', ax=axes[1])
    axes[1].set_title(f'Confusion Matrix - Optimized {model_name}')
    axes[1].set_xlabel('Predicted Label')
    axes[1].set_ylabel('True Label')

    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_validation_set(y_true, y_pred):
    """
    Evaluate the validation set using various metrics and print the results.
    
    Parameters:
    y_true (array-like): True labels of the validation set.
    y_pred (array-like): Predicted labels of the validation set.
    """
    print("\n")
    print("📊 Evaluation metrics for validation set:")
    print(f"Validation set accuracy: {accuracy_score(y_true, y_pred):.6f}")
    print(f"Validation set precision: {precision_score(y_true, y_pred):.6f}")
    print(f"Validation set recall: {recall_score(y_true, y_pred):.6f}")
    print(f"Validation set f1 score: {f1_score(y_true, y_pred):.6f}")
    print("\nValidation set confusion matrix: \n", confusion_matrix(y_true, y_pred))

    # Print the classification report for the validation set
    print("\nValidation set classification report: \n", classification_report(y_vald, y_pred))

In [ ]:
def plot_roc_curve(y_true, model, model_short_name):
    """
    Plot ROC curve for a model.
    
    Parameters:
    y_true (array-like): True labels of the validation set.
    model (object): Trained model.
    model_short_name (str): Short name of the model for labeling the plot.
    """
    
    # Get probability predictions for class 1
    y_proba = model.predict_proba(X_vald)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve - Optimized {model_short_name}')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.show()

In [ ]:
def plot_model_comparison(y_vald, y_pred_default, y_pred_optimized, model_name, model_short_name):
    """
    Plot the model comparison for the validation set
    """
    # Prepare data for the bar graph
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    default_scores = [
        accuracy_score(y_vald, y_pred_default),
        precision_score(y_vald, y_pred_default),
        recall_score(y_vald, y_pred_default),
        f1_score(y_vald, y_pred_default)
    ]
    optimized_scores = [
        accuracy_score(y_vald, y_pred_optimized),
        precision_score(y_vald, y_pred_optimized),
        recall_score(y_vald, y_pred_optimized),
        f1_score(y_vald, y_pred_optimized)
    ]

    # Plot the bar graph
    x = np.arange(len(metrics))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, default_scores, width, label='Default ' + model_short_name)
    bars2 = ax.bar(x + width/2, optimized_scores, width, label='Optimized ' + model_short_name)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xlabel('Metrics')
    ax.set_ylabel('Scores')
    ax.set_title('Performance Comparison of Default and Optimized ' + model_name)
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()

    # Annotate bars with their values
    for bar in bars1:
            height = bar.get_height()
            ax.annotate(f'{height:.6f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars2:
            height = bar.get_height()
            ax.annotate(f'{height:.6f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

##### Support Vector Machine (SVM)

An instance of SVM with default parameters

In [ ]:
# Initialize the Support Vector Machine
svm_default_model = SVC()

# Fit the Support Vector Machine on the training set
svm_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_svm_default_model_pred_val = svm_default_model.predict(X_vald)

# Print the default parameters of the Support Vector Machine
print("Default parameters of Support Vector Machine: \n", svm_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_svm_default_model_pred_val)

An instance of SVM linear kernel

In [ ]:
# Initialize the SVM linear model
svm_linear_default_model = SVC(kernel='linear', C=1)

# Fit the SVM linear model on the training set
svm_linear_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_svm_linear_default_model_pred_val = svm_linear_default_model.predict(X_vald)

# Print the default parameters of the SVM linear model
print("Default parameters of SVM linear model: \n", svm_linear_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_svm_linear_default_model_pred_val)

An instance of SVM Radial Basis Function (RBF) kernel

In [ ]:
# Initialize the SVM RBF model
svm_rbf_default_model = SVC(kernel='rbf', C=10, gamma=0.00001)

# Fit the SVM RBF model on the training set
svm_rbf_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_svm_rbf_default_model_pred_val = svm_rbf_default_model.predict(X_vald)

# Print the default parameters of the SVM RBF model
print("Default parameters of SVM RBF model: \n", svm_rbf_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_svm_rbf_default_model_pred_val)

An instance of SVM sigmoid kernel

In [ ]:
# Initialize the SVM sigmoid model
svm_sigmoid_default_model = SVC(kernel='sigmoid', C=1, gamma=0.1, coef0=1)

# Fit the SVM sigmoid model on the training set
svm_sigmoid_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_svm_sigmoid_default_model_pred_val = svm_sigmoid_default_model.predict(X_vald)

# Print the default parameters of the SVM sigmoid model
print("Default parameters of SVM sigmoid model: \n", svm_sigmoid_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_svm_sigmoid_default_model_pred_val)

Hyperparameter tuning using GridSearchCV for SVM with linear, RBF, sigmoid kernel

In [ ]:
# Set the hyperparameter grid for the SVM model
svm_param_grid = {
    'C': [0.1, 1, 5, 10],
    'coef0': [0, 0.1, 0.5],
    'gamma': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
}

# Initialize the base SVM model
svm = SVC(
    probability=True,  # Enable probability estimates for ROC curve
)

# Set up GridSearchCV
svm_grid_search = GridSearchCV(
    estimator=svm,
    param_grid=svm_param_grid,
    scoring="recall", # Choose to refit the base model based on recall score
    refit="recall",
    cv=cv,
    verbose=2,
    n_jobs=-1,
    error_score=0,
)

# Fit the grid search to training data
svm_grid_results = svm_grid_search.fit(X_train, y_train)

# Output best hyperparameters and best cross-validation score
print(f"✅ Best hyperparameters found: {svm_grid_results.best_params_}")
print(f"✅ Best cross-validation score: {svm_grid_results.best_score_:.6f}")

# Get the best model
svm_best_model = svm_grid_results.best_estimator_

# Make predictions on the validation set
y_svm_best_pred_val = svm_best_model.predict(X_vald)

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_svm_best_pred_val)

In [ ]:
# Convert the results of the GridSearchCV into a DataFrame
# Sort the DataFrame by the mean test score in descending order
# Display the top 10 configurations with the highest mean test scores
results_svm_df = pd.DataFrame(svm_grid_search.cv_results_)
results_svm_df.sort_values(
    by=['rank_test_score', 'mean_test_score'], 
    ascending=[True, False]
).head(10)

In [ ]:
results_svm_df[['params', 'mean_test_score', 'rank_test_score']].sort_values(
    by=['rank_test_score', 'mean_test_score'],
    ascending=[True, False]
).head(10)

In [ ]:
# Convert the results of the GridSearchCV into a DataFrame
results_svm_df = pd.DataFrame(svm_grid_search.cv_results_)

# Set up a 3x2 grid of subplots to include all visualizations
fig, axes = plt.subplots(3, 2, figsize=(20, 24))
fig.suptitle('SVM Hyperparameter Tuning Results', fontsize=16)

# Function to safely create a heatmap, handling empty dataframes
def safe_heatmap(filtered_df, x_param, y_param, ax, title):
    if filtered_df.empty:
        ax.text(0.5, 0.5, f"No data for\n{title}", 
                ha='center', va='center', fontsize=12)
        ax.set_title(title)
        return
    
    pivot = filtered_df.pivot_table(
        values='mean_test_score',
        index=f'param_{y_param}',
        columns=f'param_{x_param}'
    )
    
    if pivot.empty or pivot.size == 0:
        ax.text(0.5, 0.5, f"No data for\n{title}", 
                ha='center', va='center', fontsize=12)
        ax.set_title(title)
        return
        
    sns.heatmap(pivot, annot=True, fmt=".3f", cmap='YlGnBu', ax=ax)
    ax.set_title(title)
    ax.set_xlabel(x_param)
    ax.set_ylabel(y_param)

# 1. C vs gamma (for linear kernel)
df_filtered1 = results_svm_df[results_svm_df['param_kernel'] == 'linear']
safe_heatmap(
    df_filtered1, 
    'C', 
    'gamma', 
    axes[0, 0], 
    'C vs gamma\n(kernel=linear)'
)

# 2. C vs coef0 (for rbf kernel)
df_filtered2 = results_svm_df[results_svm_df['param_kernel'] == 'rbf']
safe_heatmap(
    df_filtered2, 
    'C', 
    'coef0', 
    axes[0, 1], 
    'C vs coef0\n(kernel=rbf)'
)

# 3. gamma vs coef0 (for rbf kernel)
df_filtered3 = results_svm_df[results_svm_df['param_kernel'] == 'rbf']
safe_heatmap(
    df_filtered3, 
    'gamma', 
    'coef0', 
    axes[1, 0], 
    'gamma vs coef0\n(kernel=rbf)'
)

# 4. C vs gamma (for rbf kernel)
df_filtered4 = results_svm_df[results_svm_df['param_kernel'] == 'rbf']
safe_heatmap(
    df_filtered4, 
    'C', 
    'gamma', 
    axes[1, 1], 
    'C vs gamma\n(kernel=rbf)'
)

# 5. C vs kernel
df_filtered5 = results_svm_df[results_svm_df['param_gamma'] == 1]
safe_heatmap(
    df_filtered5, 
    'C', 
    'kernel', 
    axes[2, 0], 
    'C vs kernel\n(gamma=1)'
)

# 6. Bar chart of kernel performance
kernel_scores = results_svm_df.groupby('param_kernel')['mean_test_score'].mean()
sns.barplot(x=kernel_scores.index, y=kernel_scores.values, ax=axes[2, 1])
axes[2, 1].set_title('Average Performance by Kernel')
axes[2, 1].set_xlabel('Kernel')
axes[2, 1].set_ylabel('Mean Test Score')
for i, v in enumerate(kernel_scores.values):
    axes[2, 1].text(i, v + 0.01, f"{v:.3f}", ha='center')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# Filter for one kernel and one coef0 for simplicity
svm_subset = results_svm_df[
    (results_svm_df['param_kernel'] == 'rbf') &
    (results_svm_df['param_coef0'] == 0.0)
]

pivot = svm_subset.pivot_table(
    index='param_C',
    columns='param_gamma',
    values='mean_test_score'
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title('Recall Heatmap for RBF Kernel (coef0=0)')
plt.ylabel('C')
plt.xlabel('Gamma')
plt.show()


In [ ]:
# Track recall for different kernels across C values
for svm_kernel in svm_param_grid['kernel']:
    svm_subset = results_svm_df[
        (results_svm_df['param_kernel'] == svm_kernel) &
        (results_svm_df['param_gamma'] == 1) & 
        (results_svm_df['param_coef0'] == 0.0)
    ]
    plt.plot(svm_subset['param_C'], svm_subset['mean_test_score'], label=svm_kernel)

plt.title("Recall vs C for Different Kernels (gamma=1, coef0=0)")
plt.xlabel("C")
plt.ylabel("Mean Recall")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plot ROC curve for the optimized SVM model
plot_roc_curve(y_vald, svm_best_model, "SVM")

In [ ]:
plot_confusion_matrices_side_by_side(y_vald, y_svm_default_model_pred_val, y_svm_best_pred_val, "SVM")

In [ ]:
plot_model_comparison(y_vald, y_svm_default_model_pred_val, y_svm_best_pred_val, "Support Vector Machine", "SVM")

##### Decision Tree Classifier (DTC)

An instance of DTC with default parameters

In [ ]:
# Initialize the Decision Tree Classifier
dtc_default_model = DecisionTreeClassifier(random_state=seed_num)

# Fit the Decision Tree Classifier on the training set
dtc_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_dtc_default_model_pred_val = dtc_default_model.predict(X_vald)

# Print the default parameters of the Decision Tree Classifier
print("Default parameters of Decision Tree Classifier: \n", dtc_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_dtc_default_model_pred_val)

Hyperparameter tuning using GridSearchCV for DTC

In [ ]:
# Set the hyperparameter grid for the Decision Tree Classifier
dtc_param_grid = {
    'ccp_alpha': [0.0, 0.0005, 0.001],
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 7, 9, 11, 13],
    'max_features': ['sqrt', 'log2'],
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0, 0.001, 0.01],
    'min_samples_leaf': [1, 2, 4, 6],
    'min_samples_split': [2, 4, 6],
    'splitter': ['best'],
}

# Initialize the base Decision Tree
dtc = DecisionTreeClassifier(random_state=seed_num)

# Set up GridSearchCV
dtc_grid_search = GridSearchCV(
    estimator=dtc,
    param_grid=dtc_param_grid,
    scoring="recall", # Choose to refit the base model based on recall score
    cv=cv,
    verbose=2,
    n_jobs=-1,
    error_score=0,
)

# Fit the grid search to training data
dtc_grid_results = dtc_grid_search.fit(X_train, y_train)

# Output best hyperparameters and best cross-validation score
print(f"✅ Best hyperparameters found: {dtc_grid_results.best_params_}")
print(f"✅ Best cross-validation score: {dtc_grid_results.best_score_:.6f}")

# Get the best model
dtc_best_model = dtc_grid_results.best_estimator_

# Make predictions on the validation set
y_dtc_best_pred_val = dtc_best_model.predict(X_vald)

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_dtc_best_pred_val)


In [ ]:
# Convert the results of the GridSearchCV into a DataFrame
# Sort the DataFrame by the mean test score in descending order
# Display the top 10 configurations with the highest mean test scores
results_dtc_df = pd.DataFrame(dtc_grid_search.cv_results_)
results_dtc_df.sort_values(
    by=['rank_test_score', 'mean_test_score'], 
    ascending=[True, False]
).head(10)

In [ ]:
results_dtc_df[['params', 'mean_test_score', 'rank_test_score']].sort_values(
    by=['rank_test_score', 'mean_test_score'],
    ascending=[True, False]
).head(10)

In [ ]:
# Create a comprehensive visualization with multiple heatmaps of DTC hyperparameters tuning results
results_dtc_df = pd.DataFrame(dtc_grid_search.cv_results_)

# Set up a 3x2 grid of subplots to include all visualizations
fig, axes = plt.subplots(3, 2, figsize=(20, 24))
fig.suptitle('DTC Hyperparameter Tuning Results', fontsize=16)

# Function to safely create a heatmap, handling empty dataframes
def safe_heatmap(filtered_df, x_param, y_param, ax, title):
    if filtered_df.empty:
        ax.text(0.5, 0.5, f"No data for\n{title}", 
                ha='center', va='center', fontsize=12)
        ax.set_title(title)
        return
    
    pivot = filtered_df.pivot_table(
        values='mean_test_score',
        index=f'param_{y_param}',
        columns=f'param_{x_param}'
    )
    
    if pivot.empty or pivot.size == 0:
        ax.text(0.5, 0.5, f"No data for\n{title}", 
                ha='center', va='center', fontsize=12)
        ax.set_title(title)
        return
        
    sns.heatmap(pivot, annot=True, fmt=".3f", cmap='YlGnBu', ax=ax)
    ax.set_title(title)
    ax.set_xlabel(x_param)
    ax.set_ylabel(y_param)

# 1. max_depth vs min_samples_leaf
df_filtered1 = results_dtc_df[
    (results_dtc_df['param_criterion'] == 'gini') &
    (results_dtc_df['param_ccp_alpha'] == 0.0005)
]
safe_heatmap(
    df_filtered1, 
    'min_samples_leaf', 
    'max_depth', 
    axes[0, 0], 
    'max_depth vs min_samples_leaf\n(criterion=gini, ccp_alpha=0.0005)'
)

# 2. max_depth vs criterion
df_filtered2 = results_dtc_df[
    (results_dtc_df['param_min_samples_leaf'] == 2) &
    (results_dtc_df['param_ccp_alpha'] == 0.0005)
]
safe_heatmap(
    df_filtered2, 
    'criterion', 
    'max_depth', 
    axes[0, 1], 
    'max_depth vs criterion\n(min_samples_leaf=2, ccp_alpha=0.0005)'
)

# 3. max_depth vs ccp_alpha
df_filtered3 = results_dtc_df[
    (results_dtc_df['param_criterion'] == 'gini') &
    (results_dtc_df['param_min_samples_leaf'] == 2)
]
safe_heatmap(
    df_filtered3, 
    'ccp_alpha', 
    'max_depth', 
    axes[1, 0], 
    'max_depth vs ccp_alpha\n(criterion=gini, min_samples_leaf=2)'
)

# 4. max_depth vs max_features
df_filtered4 = results_dtc_df[
    (results_dtc_df['param_criterion'] == 'gini') &
    (results_dtc_df['param_min_samples_leaf'] == 2) &
    (results_dtc_df['param_ccp_alpha'] == 0.0005)
]
safe_heatmap(
    df_filtered4, 
    'max_features', 
    'max_depth', 
    axes[1, 1], 
    'max_depth vs max_features\n(criterion=gini, min_samples_leaf=2)'
)

# 5. max_features vs min_impurity_decrease heatmap
df_filtered5 = results_dtc_df[
    (results_dtc_df['param_criterion'] == 'gini') &
    (results_dtc_df['param_max_depth'] == 13) &
    (results_dtc_df['param_min_samples_leaf'] == 2)
]

if not df_filtered5.empty:
    pivot5 = df_filtered5.pivot_table(
        values='mean_test_score',
        index='param_max_features',
        columns='param_min_impurity_decrease'
    )
    
    if not pivot5.empty and pivot5.size > 0:
        sns.heatmap(pivot5, annot=True, fmt=".3f", cmap='YlGnBu', ax=axes[2, 0])
        axes[2, 0].set_title('max_features vs min_impurity_decrease\n(criterion=gini, max_depth=13, min_samples_leaf=2)')
        axes[2, 0].set_xlabel('min_impurity_decrease')
        axes[2, 0].set_ylabel('max_features')
    else:
        axes[2, 0].text(0.5, 0.5, "No data available for\nmax_features vs min_impurity_decrease heatmap", 
                    ha='center', va='center', fontsize=12)
        axes[2, 0].set_title('max_features vs min_impurity_decrease')
else:
    axes[2, 0].text(0.5, 0.5, "No data available for\nmax_features vs min_impurity_decrease heatmap", 
                ha='center', va='center', fontsize=12)
    axes[2, 0].set_title('max_features vs min_impurity_decrease')

# 6. Bar chart of max_features performance
max_features_scores = results_dtc_df.groupby('param_max_features')['mean_test_score'].mean()
sns.barplot(x=max_features_scores.index, y=max_features_scores.values, ax=axes[2, 1])
axes[2, 1].set_title('Average Performance by max_features Value')
axes[2, 1].set_xlabel('max_features')
axes[2, 1].set_ylabel('Mean Test Score')
for i, v in enumerate(max_features_scores.values):
    axes[2, 1].text(i, v + 0.01, f"{v:.3f}", ha='center')

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# Plot ROC curve for the optimized DTC model
plot_roc_curve(y_vald, dtc_best_model, "DTC")

In [ ]:
# Print the tree as text
print(export_text(dtc_best_model, feature_names=list(X_train_df.columns)))

# Plot the tree
plt.figure(figsize=(30, 20))
plot_tree(dtc_best_model, feature_names=list(X_train_df.columns), filled=True, rounded=True, fontsize=10, max_depth=3)
plt.title("Optiimized DTC Visualization")
plt.show()

In [ ]:
# Label the features with their importance scores
feature_importances = pd.DataFrame({
    "feature": selected_features,
    "importance": dtc_best_model.feature_importances_
})
feature_importances = feature_importances.sort_values("importance", ascending=False)

# Label the bar with value
plt.figure(figsize=(12, max(6, len(feature_importances) * 0.3)))  # dynamic height
sns.barplot(x="importance", y="feature", data=feature_importances)
for index, value in enumerate(feature_importances['importance']):
    plt.text(value, index, str(round(value, 6)))
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance (Optimized DTC)")
plt.tight_layout()
plt.show()


In [ ]:
plot_confusion_matrices_side_by_side(y_vald, y_dtc_default_model_pred_val, y_dtc_best_pred_val, "DTC")

In [ ]:
plot_model_comparison(y_vald, y_dtc_default_model_pred_val, y_dtc_best_pred_val, "Decision Tree Classifier", "DTC")

##### K-Nearest Neighbour (KNN) 

An instance of KNN Classifier with default parameters

In [ ]:
# Initialize the K-Nearest Neighbors Classifier
knn_default_model = KNeighborsClassifier()

# Fit the K-Nearest Neighbors Classifier on the training set
knn_default_model.fit(X_train, y_train)

# Make predictions on the validation set
y_knn_default_model_pred_val = knn_default_model.predict(X_vald)

# Print the default parameters of the K-Nearest Neighbors Classifier
print("Default parameters of K-Nearest Neighbors Classifier: \n", knn_default_model.get_params())

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_knn_default_model_pred_val)

Hyperparameter tuning using GridSearchCV for KNN Classifier

In [ ]:
# Set the hyperparameter grid for the K-Nearest Neighbor Classifier
# {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
knn_param_grid = {
    "algorithm": ["auto", "ball_tree", "kd_tree"],
    "leaf_size": range(10, 50, 5),
    "metric": ["euclidean", "manhattan", "minkowski"],
    "n_neighbors": range(3, 30, 2),
    "weights": ["uniform", "distance"],
}

# Initialize the base K-Nearest Neighbors Classifier
knn = KNeighborsClassifier()

# Set up GridSearchCV
knn_grid_search = GridSearchCV(
    estimator=knn,
    param_grid=knn_param_grid,
    scoring="recall", # Choose to refit the base model based on recall score
    cv=cv, # Repeated Stratified K-Fold Cross Validation
    verbose=2,
    n_jobs=-1,
    error_score=0,
)

# Fit the grid search to training data
knn_grid_results = knn_grid_search.fit(X_train, y_train)

# Output best parameters and score
print(f"✅ Best hyperparameters found: {knn_grid_results.best_params_}")
print(f"✅ Best cross-validation score: {knn_grid_results.best_score_:.6f}")

# Get the best model
knn_best_model = knn_grid_results.best_estimator_

# Make predictions on the validation set
y_knn_best_pred_val = knn_best_model.predict(X_vald)

# Print the evaluation metrics for the validation set
evaluate_validation_set(y_vald, y_knn_best_pred_val)


In [ ]:
# Convert the results of the GridSearchCV into a DataFrame
# Sort the DataFrame by the mean test score in descending order
# Display the top 10 configurations with the highest mean test scores
results_knn_df = pd.DataFrame(knn_grid_search.cv_results_)
results_knn_df.sort_values(
    by=['rank_test_score', 'mean_test_score'], 
    ascending=[True, False]
).head(10)

In [ ]:
results_knn_df[['params', 'mean_test_score', 'rank_test_score']].sort_values(
    by=['rank_test_score', 'mean_test_score'],
    ascending=[True, False]
).head(10)

In [ ]:
# Create a comprehensive visualization with multiple heatmaps of KNN hyperparameters tuning results
results_knn_df = pd.DataFrame(knn_grid_search.cv_results_)

# Set up a 2x2 grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('KNN Hyperparameter Tuning Results', fontsize=16)

# 1. n_neighbors vs leaf_size (for auto algorithm, euclidean metric, distance weights)
df_filtered1 = results_knn_df[
    (results_knn_df['param_algorithm'] == 'auto') &
    (results_knn_df['param_metric'] == 'euclidean') &
    (results_knn_df['param_weights'] == 'distance')
]
pivot1 = df_filtered1.pivot_table(
    values='mean_test_score',
    index='param_n_neighbors',
    columns='param_leaf_size'
)
sns.heatmap(pivot1, annot=True, fmt=".3f", cmap='YlGnBu', ax=axes[0, 0])
axes[0, 0].set_title('n_neighbors vs leaf_size\n(algorithm=auto, metric=euclidean, weights=distance)')
axes[0, 0].set_xlabel('leaf_size')
axes[0, 0].set_ylabel('n_neighbors')

# 2. n_neighbors vs metric (for auto algorithm, leaf_size=30, distance weights)
df_filtered2 = results_knn_df[
    (results_knn_df['param_algorithm'] == 'auto') &
    (results_knn_df['param_leaf_size'] == 30) &
    (results_knn_df['param_weights'] == 'distance')
]
pivot2 = df_filtered2.pivot_table(
    values='mean_test_score',
    index='param_n_neighbors',
    columns='param_metric'
)
sns.heatmap(pivot2, annot=True, fmt=".3f", cmap='YlGnBu', ax=axes[0, 1])
axes[0, 1].set_title('n_neighbors vs metric\n(algorithm=auto, leaf_size=30, weights=distance)')
axes[0, 1].set_xlabel('metric')
axes[0, 1].set_ylabel('n_neighbors')

# 3. n_neighbors vs algorithm (for euclidean metric, leaf_size=30, distance weights)
df_filtered3 = results_knn_df[
    (results_knn_df['param_metric'] == 'euclidean') &
    (results_knn_df['param_leaf_size'] == 30) &
    (results_knn_df['param_weights'] == 'distance')
]
pivot3 = df_filtered3.pivot_table(
    values='mean_test_score',
    index='param_n_neighbors',
    columns='param_algorithm'
)
sns.heatmap(pivot3, annot=True, fmt=".3f", cmap='YlGnBu', ax=axes[1, 0])
axes[1, 0].set_title('n_neighbors vs algorithm\n(metric=euclidean, leaf_size=30, weights=distance)')
axes[1, 0].set_xlabel('algorithm')
axes[1, 0].set_ylabel('n_neighbors')

# 4. n_neighbors vs weights (for auto algorithm, euclidean metric, leaf_size=30)
df_filtered4 = results_knn_df[
    (results_knn_df['param_algorithm'] == 'auto') &
    (results_knn_df['param_metric'] == 'euclidean') &
    (results_knn_df['param_leaf_size'] == 30)
]
pivot4 = df_filtered4.pivot_table(
    values='mean_test_score',
    index='param_n_neighbors',
    columns='param_weights'
)
sns.heatmap(pivot4, annot=True, fmt=".3f", cmap='YlGnBu', ax=axes[1, 1])
axes[1, 1].set_title('n_neighbors vs weights\n(algorithm=auto, metric=euclidean, leaf_size=30)')
axes[1, 1].set_xlabel('weights')
axes[1, 1].set_ylabel('n_neighbors')

plt.tight_layout(rect=[0, 0, 1, 0.97])  # Adjust layout to make room for suptitle
plt.show()

**Principal Component Analysis (PCA)** transforms the original features into a new set of uncorrelated components, which are linear combinations of the original variables. These components are ordered based on the amount of variance they capture from the data. PCA reduces dimensionality while preserving as much of the underlying data structure as possible.

In this case, PCA is applied to reduce the original eight-dimensional feature space to two principal components, specifically for visualization and plotting purposes.

In [ ]:
# Apply PCA to validation features
# Reduced to 2D for visualization
pca = PCA(n_components=2)

# Fit PCA on the validation set
X_pca = pca.fit_transform(X_vald)

In [ ]:
# Plot the true labels in a separate graph
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=y_vald, palette='Set1', s=40)
plt.title("True Labels Scatter Plot (PCA-reduced 2D)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="True Class")
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=True, sharey=True)

# Default KNN
sns.scatterplot(ax=axes[0], x=X_pca[:, 0], y=X_pca[:, 1], hue=y_knn_default_model_pred_val, palette='Set1', s=40)
axes[0].set_title("Default KNN Prediction Scatter Plot (PCA-reduced 2D)")
axes[0].set_xlabel("Principal Component 1")
axes[0].set_ylabel("Principal Component 2")
axes[0].legend(title="Predicted Class")
axes[0].grid(True)

# Optimized KNN
sns.scatterplot(ax=axes[1], x=X_pca[:, 0], y=X_pca[:, 1], hue=y_knn_best_pred_val, palette='Set1', s=40)
axes[1].set_title("Optimized KNN Prediction Scatter Plot (PCA-reduced 2D)")
axes[1].set_xlabel("Principal Component 1")
axes[1].set_ylabel("Principal Component 2")
axes[1].legend(title="Predicted Class")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Compare predictions
y_pred_default = knn_default_model.predict(X_vald)
y_pred_optimized = knn_best_model.predict(X_vald)

# Identify points where predictions differ
diff_mask = y_pred_default != y_pred_optimized
diff_labels = pd.Series(diff_mask).map({True: 'Yes', False: 'No'})

# PCA transform
X_pca = PCA(n_components=2).fit_transform(X_vald)

# Plot only the changed predictions
plt.figure(figsize=(8, 5))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=diff_labels, palette='deep', s=40)
plt.title("Points with Different Predictions (Default vs Optimized KNN)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Prediction Changed")
plt.grid(True)
plt.show()

In [ ]:
probs = knn_best_model.predict_proba(X_vald)[:, 1]
plt.figure(figsize=(8, 5))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=probs, palette='coolwarm', s=40)
plt.title("Probability of Class 1 (Optimized KNN)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.show()

In [ ]:
plot_confusion_matrices_side_by_side(y_vald, y_knn_default_model_pred_val, y_knn_best_pred_val, "KNN")

In [ ]:
plot_model_comparison(y_vald, y_knn_default_model_pred_val, y_knn_best_pred_val, "K-Nearest Neighbors Classifier", "KNN")

#### Evaluate the models
Perform a comparison between the predictive models. <br>
Report the accuracy, recall, precision and F1-score measures as well as the confusion matrix if it is a classification problem. <br>
Report the R2 score, mean squared error and mean absolute error if it is a regression problem.
______________________________________________________________________________________
Description:

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """
    Evaluate a model using multiple metrics (weighted and macro)
    """
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Weighted Precision': precision_score(y_true, y_pred, average='weighted'),
        'Weighted Recall': recall_score(y_true, y_pred, average='weighted'),
        'Weighted F1 Score': f1_score(y_true, y_pred, average='weighted'),
        'Macro Precision': precision_score(y_true, y_pred, average='macro'),
        'Macro Recall': recall_score(y_true, y_pred, average='macro'),
        'Macro F1 Score': f1_score(y_true, y_pred, average='macro')
    }
    return metrics

In [ ]:
def compare_models(y_true, model1_pred, model2_pred, model3_pred, 
                   model1_name, model2_name, model3_name):
    """
    Compare three models using various metrics
    """
    # Calculate metrics for each model
    metrics1 = evaluate_model(y_true, model1_pred, model1_name)
    metrics2 = evaluate_model(y_true, model2_pred, model2_name)
    metrics3 = evaluate_model(y_true, model3_pred, model3_name)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame([metrics1, metrics2, metrics3])

    cm_model1 = confusion_matrix(y_true, model1_pred)
    cm_model2 = confusion_matrix(y_true, model2_pred)
    cm_model3 = confusion_matrix(y_true, model3_pred)

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    # Confusion matrix for model 1
    sns.heatmap(cm_model1, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title(f'Confusion Matrix - Optimized ' + model1_name)
    axes[0].set_xlabel('Predicted Label')
    axes[0].set_ylabel('True Label')
    
    # Confusion matrix for model 2
    sns.heatmap(cm_model2, annot=True, fmt='d', cmap='Blues', ax=axes[1])
    axes[1].set_title(f'Confusion Matrix - Optimized ' + model2_name)
    axes[1].set_xlabel('Predicted Label')
    axes[1].set_ylabel('True Label')

    # Confusion matrix for model 3
    sns.heatmap(cm_model3, annot=True, fmt='d', cmap='Blues', ax=axes[2])
    axes[2].set_title(f'Confusion Matrix - Optimized ' + model3_name)
    axes[2].set_xlabel('Predicted Label')
    axes[2].set_ylabel('True Label')

    plt.tight_layout()
    plt.show()
    
    # Plot comparison bar chart - Accuracy, Weighted Precision, Weighted Recall, Weighted F1 Score
    metrics_to_plot = ['Accuracy', 'Weighted Precision', 'Weighted Recall', 'Weighted F1 Score']
    plt.figure(figsize=(12, 6))
    x = np.arange(len(metrics_to_plot))
    width = 0.25
    
    bars1 = plt.bar(x - width, comparison_df[metrics_to_plot].iloc[0], width, label=model1_name)
    bars2 = plt.bar(x, comparison_df[metrics_to_plot].iloc[1], width, label=model2_name)
    bars3 = plt.bar(x + width, comparison_df[metrics_to_plot].iloc[2], width, label=model3_name)

    # Annotate bars with their values
    for bar in bars1:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars2:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars3:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('Model Comparison (Weighted Avg)')
    plt.xticks(x, metrics_to_plot)
    plt.legend()
    plt.show()

    # Plot comparison bar chart - Accuracy, Macro Precision, Macro Recall, Macro F1 Score
    metrics_to_plot = ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1 Score']
    plt.figure(figsize=(12, 6))
    x = np.arange(len(metrics_to_plot))
    width = 0.25
    
    bars1 = plt.bar(x - width, comparison_df[metrics_to_plot].iloc[0], width, label=model1_name)
    bars2 = plt.bar(x, comparison_df[metrics_to_plot].iloc[1], width, label=model2_name)
    bars3 = plt.bar(x + width, comparison_df[metrics_to_plot].iloc[2], width, label=model3_name)

    # Annotate bars with their values
    for bar in bars1:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars2:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars3:
        height = bar.get_height()
        plt.annotate(f'{height:.4f}', xy=(bar.get_x() + bar.get_width() / 2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('Model Comparison (Macro Avg)')
    plt.xticks(x, metrics_to_plot)
    plt.legend()
    plt.show()
    
    # Print detailed classification reports
    print(f"\nClassification Report for {model1_name}:")
    print(classification_report(y_true, model1_pred))
    print(f"\nClassification Report for {model2_name}:")
    print(classification_report(y_true, model2_pred))
    print(f"\nClassification Report for {model3_name}:")
    print(classification_report(y_true, model3_pred))
    
    return comparison_df

##### Support Vector Machine (SVM)

In [ ]:
y_pred_svm = svm_best_model.predict(X_test)

##### Decision Tree Classifier (DTC)

In [ ]:
y_pred_dtc = dtc_best_model.predict(X_test)

##### K-Nearest Neighbour (KNN)

In [ ]:
y_pred_knn = knn_best_model.predict(X_test)

#### Model Comparison

In [ ]:
results = compare_models(
    y_test,
    y_pred_svm,
    y_pred_dtc,
    y_pred_knn,
    model1_name="SVM",
    model2_name="DTC",
    model3_name="KNN"
    )

In [ ]:
# Print detailed results
print("\nDetailed Model Comparison:")
print(results.to_string(index=False))

Conclusion:
